In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Read the data
df = pd.read_csv('unemployment_rate_cleaned.csv', index_col=0)

# Select representative countries from each continent
selected_countries = {
    'North America': 'United States',
    'Europe': 'Germany',
    'Asia': 'Japan',
    'Oceania': 'Australia'
}

# Create a list of the selected country names
country_list = list(selected_countries.values())

# Convert index to numeric for proper filtering
df.index = pd.to_numeric(df.index)

# Focus on the COVID period and surrounding years (2018-2022)
covid_period = df[
    (df.index >= 2018) & 
    (df.index <= 2022)
][country_list]

print("Data shape:", covid_period.shape)
print("\nFirst few rows of the selected data:")
print(covid_period.head())

Data shape: (5, 4)

First few rows of the selected data:
        United States   Germany     Japan  Australia
2018.0       3.891667  3.207761  2.441806   5.307348
2019.0       3.675000  2.977034  2.349567   5.168685
2020.0       8.091667  3.629752  2.776169   6.462792
2021.0       5.350000  3.566733  2.817593   5.127467
2022.0       3.633333  3.065084  2.600749   3.716131


In [7]:
def create_covid_trend_visualization(data):
    fig = go.Figure()
    
    # Create a line for each country with distinct colors
    colors = {
        'United States': '#1f77b4',  # Blue
        'Germany': '#ff7f0e',        # Orange
        'Japan': '#2ca02c',          # Green
        'Australia': '#d62728'       # Red
    }
    
    for country in data.columns:
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data[country],
                name=country,
                line=dict(
                    color=colors[country],
                    width=3
                ),
                mode='lines+markers',
                marker=dict(size=8),
                hovertemplate=(
                    f"{country}<br>" +
                    "Year: %{x}<br>" +
                    "Unemployment Rate: %{y:.2f}%<extra></extra>"
                )
            )
        )
    
    # Add COVID period highlighting
    fig.add_vrect(
        x0=2020, x1=2021,
        fillcolor="rgba(128, 128, 128, 0.2)",
        layer="below", line_width=0,
        annotation=dict(
            text="COVID-19 Period",
            font=dict(size=14),
            showarrow=False,
            x=2020.5,
            y=1.05,
            yref="paper"
        )
    )
    
    # Update layout for better presentation
    fig.update_layout(
        title={
            'text': 'Impact of COVID-19 on Unemployment Rates (2018-2022)',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20)
        },
        xaxis_title="Year",
        yaxis_title="Unemployment Rate (%)",
        hovermode='x unified',
        template='plotly_white',
        height=600,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255, 255, 255, 0.8)"
        )
    )
    
    return fig

trend_fig = create_covid_trend_visualization(covid_period)
trend_fig.write_html('covid_unemployment_trends.html')

In [9]:
def analyze_covid_impact(data):
    # Calculate percentage changes during COVID
    pre_covid = data.loc[2018:2019].mean()
    during_covid = data.loc[2020:2021].mean()
    post_covid = data.loc[2022:2022].mean()
    
    # Create comparison DataFrame
    comparison = pd.DataFrame({
        'Pre-COVID (2018-2019)': pre_covid,
        'During COVID (2020-2021)': during_covid,
        'Post-COVID (2022)': post_covid
    })
    
    # Calculate percentage changes
    comparison['COVID Impact (%)'] = (
        (comparison['During COVID (2020-2021)'] - comparison['Pre-COVID (2018-2019)']) / 
        comparison['Pre-COVID (2018-2019)'] * 100
    )
    
    # Create visualization of the impact
    fig = go.Figure()
    
    # Add bar chart for percentage changes
    fig.add_trace(go.Bar(
        x=comparison.index,
        y=comparison['COVID Impact (%)'],
        text=comparison['COVID Impact (%)'].round(1).astype(str) + '%',
        textposition='auto',
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    ))
    
    fig.update_layout(
        title='Percentage Change in Unemployment During COVID-19',
        yaxis_title='Change in Unemployment Rate (%)',
        template='plotly_white',
        height=500,
        showlegend=False
    )
    
    return comparison, fig

impact_data, impact_fig = analyze_covid_impact(covid_period)
impact_fig.write_html('covid_impact_analysis.html')

# Print detailed analysis
print("\nDetailed COVID-19 Impact Analysis:")
print("-" * 50)
print(impact_data.round(2))


Detailed COVID-19 Impact Analysis:
--------------------------------------------------
               Pre-COVID (2018-2019)  During COVID (2020-2021)  \
United States                   3.78                      6.72   
Germany                         3.09                      3.60   
Japan                           2.40                      2.80   
Australia                       5.24                      5.80   

               Post-COVID (2022)  COVID Impact (%)  
United States               3.63             77.64  
Germany                     3.07             16.36  
Japan                       2.60             16.75  
Australia                   3.72             10.64  


In [11]:
def create_period_distribution_plot(data):
    fig = go.Figure()
    
    periods = {
        'Pre-COVID': data.loc[2018:2019],
        'During COVID': data.loc[2020:2021],
        'Post-COVID': data.loc[2022:2022]
    }
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for (period, period_data), color in zip(periods.items(), colors):
        fig.add_trace(go.Box(
            y=period_data.values.flatten(),
            name=period,
            marker_color=color,
            boxpoints='all',
            jitter=0.3,
            pointpos=-1.8
        ))
    
    fig.update_layout(
        title='Distribution of Unemployment Rates Across COVID-19 Periods',
        yaxis_title='Unemployment Rate (%)',
        template='plotly_white',
        height=500,
        showlegend=False
    )
    
    return fig

dist_fig = create_period_distribution_plot(covid_period)
dist_fig.write_html('covid_period_distribution.html')

In [13]:
def create_crisis_comparison_visualization(data):
    fig = go.Figure()
    
    # Add lines for each country
    for country, color in crisis_countries.items():
        fig.add_trace(
            go.Scatter(
                # Use data.index instead of x directly
                x=data.index,  # This is the correct way to reference the years
                y=data[country],
                name=country,
                line=dict(color=color, width=3),
                mode='lines+markers',
                marker=dict(size=8),
                # Fixed the hovertemplate syntax
                hovertemplate=(
                    f"{country}<br>" +
                    "Year: %{x}<br>" +
                    "Unemployment: %{y:.2f}%<extra></extra>"
                )
            )
        )
    
    # Add rectangles highlighting crisis periods - using proper year values
    fig.add_vrect(
        x0=2008, x1=2010,  # Using numeric values directly for the years
        fillcolor="rgba(255, 0, 0, 0.1)",
        layer="below", 
        line_width=0,
        annotation=dict(
            text="Global Financial Crisis",
            font=dict(size=14),
            showarrow=False,
            x=2009,
            y=1.05,
            yref="paper"
        )
    )
    
    fig.update_layout(
        title={
            'text': 'Unemployment Rates During Major Economic Crises (2006-2022)',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="Year",
        yaxis_title="Unemployment Rate (%)",
        hovermode='x unified',
        template='plotly_white',
        height=600,
        showlegend=True
    )
    
    return fig